<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural ---- Desafio 2
## Custom embedddings con Gensim


- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec
#librerias para preprocesamiento
import urllib.request
import re

### Datos
Utilizaremos la obra el retrato de Dorian Gray de Oscar Wilde tomado del sitio recomendado www.texto.info

In [2]:
url = "https://www.textos.info/oscar-wilde/el-retrato-de-dorian-gray/pdf"
urllib.request.urlretrieve(url, "el-retrato-de-dorian-gray.pdf")
print("Archivo PDF descargado exitosamente.")

Archivo PDF descargado exitosamente.


In [3]:
#Libraria para manejo de archivos pdf
#!pip install pymupdf

In [4]:
#convertir archivo pfd a texto
import fitz
def pdf_to_txt(pdf_path, txt_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page in pdf_document:
        text += page.get_text()
    with open(txt_path, "w", encoding="utf-8") as txt_file:
        txt_file.write(text)

    print(f"Archivo convertido a TXT almacenado en {txt_path}")
pdf_to_txt("el-retrato-de-dorian-gray.pdf", "el-retrato-de-dorian-gray.txt")  # se genera nuevo archivo modificado con la limpieza y normalizacion del texto

Archivo convertido a TXT almacenado en el-retrato-de-dorian-gray.txt


In [3]:
# Leer el archivo de texto
with open("el-retrato-de-dorian-gray.txt", "r", encoding="utf-8") as file:
    text = file.read()
# Normalizacion de  texto a minusculas, eliminacion de caracterias diferentes a alfabeticos
text = text.lower()
text = re.sub(r'[^a-záéíóúüñ\s]', '', text)
text = re.sub(r'[!?]', '', text)  
text = re.sub(r' +', ' ', text)
archivo=open("el-retrato-de-dorian-gray_mod.txt", "w", encoding="utf-8") 
archivo.write(text)
archivo.close()

In [4]:
# Armar el dataset 
df = pd.read_fwf('el-retrato-de-dorian-gray_mod.txt')
df.head(15)

,el retrato de dorian gray
0,oscar wilde
1,textosinfo
2,biblioteca digital abierta
3,texto núm
4,título el retrato de dorian gray
5,autor oscar wilde
6,etiquetas novela
7,editor edu robsy
8,fecha de creación de mayo de
9,fecha de modificación de noviembre de


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 7337


In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:10]

[['oscar', 'wilde'],
 ['textosinfo'],
 ['biblioteca', 'digital', 'abierta'],
 ['texto', 'núm'],
 ['título', 'el', 'retrato', 'de', 'dorian', 'gray'],
 ['autor', 'oscar', 'wilde'],
 ['etiquetas', 'novela'],
 ['editor', 'edu', 'robsy'],
 ['fecha', 'de', 'creación', 'de', 'mayo', 'de'],
 ['fecha', 'de', 'modificación', 'de', 'noviembre', 'de']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 7337


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1743


### 3 - Entrenar embeddings

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 568821.4375
Loss after epoch 1: 405794.4375
Loss after epoch 2: 360299.75
Loss after epoch 3: 348354.5
Loss after epoch 4: 343603.125
Loss after epoch 5: 303226.25
Loss after epoch 6: 293687.0
Loss after epoch 7: 291420.0
Loss after epoch 8: 288855.25
Loss after epoch 9: 289779.0
Loss after epoch 10: 286186.0
Loss after epoch 11: 284928.75
Loss after epoch 12: 276077.0
Loss after epoch 13: 267276.0
Loss after epoch 14: 266699.5
Loss after epoch 15: 263916.5
Loss after epoch 16: 263619.0
Loss after epoch 17: 263014.5
Loss after epoch 18: 262642.5
Loss after epoch 19: 261445.0


(845798, 1570120)

### 4 - Ensayar

In [23]:
palabras = ["retrato", "artista","espejos", "retrato", "belleza","hermosa", "juventud", "asesinato"]#, "imperfect", "word", "sound", "ideas", "mind", "syntax", "similar", "meaning", "teutonic", "vowels", "influence"]
for palabra in palabras:
    similar_words = w2v_model.wv.most_similar(positive=[palabra], topn=10)
    print(f"Most similar words for '{palabra}':")
    for similar_word, similarity_score in similar_words:
        print(f"- {similar_word}: {similarity_score}")
    print()

Most similar words for 'retrato':
- pintado: 0.8089762926101685
- exponer: 0.8019140958786011
- lienzo: 0.789556086063385
- examinar: 0.783231258392334
- matrimonio: 0.7800175547599792
- daría: 0.7754347920417786
- hermoso: 0.7750706672668457
- verdadero: 0.760972261428833
- tuyo: 0.7587563395500183
- cambio: 0.7555050253868103

Most similar words for 'artista':
- hermoso: 0.9059011340141296
- matrimonio: 0.8793865442276001
- suyo: 0.8784503936767578
- problema: 0.8635460138320923
- encanto: 0.8626889586448669
- genio: 0.8569450378417969
- resto: 0.855726420879364
- asunto: 0.85463947057724
- caballero: 0.8460862040519714
- capricho: 0.8449692726135254

Most similar words for 'espejos':
- brillantes: 0.9693781733512878
- verdes: 0.9684475660324097
- criados: 0.9647932648658752
- curiosos: 0.9626385569572449
- bordados: 0.9614115357398987
- negros: 0.9604668617248535
- cristales: 0.9574335217475891
- paredes: 0.9567935466766357
- delicados: 0.9543972015380859
- hacían: 0.953052222728729

Se analizaron algunas palabras relevantes de la obra y en general y tomando en cuenta el contexto tienen mucho sentido por ejemplo: 
retrato con (lienzo,pintado), asesinato con (culpa, delito, accidente), espejos con (brillantes, cristales).
Ademas se asocian el contenido del libro que se centra en la obsecion de la belleza y juventud (que precisamente estan dentro de sus palabras similares)
Hay algunos casos donde las palabras no se oberva similaridad pero a nivel global se observa un buen desempeño.

### 5 - Visualizar agrupación de vectores

In [24]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [26]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()
#fig.show(renderer="colab") # esto para plotly en colab

Se puede observar en el grafico, cercania entre palabras por ejemplo: (mujer, duquesa), (exclamó, preguntó,murmuró, respondió), (sus, ojos,manos, labios), retrato, cuadro, rostro) diversas palabras que se utilizan de manera conjunta dentro de la obra

In [27]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab
#fig.show(renderer="colab") # esto para plotly en colab

En este grafico 3D se observan otras relaciones relevantes al contenido de la obra, la cercania entre los nombres de los personajes principales( dorian, hallward, basil, sibyl), entre otras palabras similares (mucho y demasiado)
